<a href="https://colab.research.google.com/github/ZetaRR/NY-apartments-EDA-Regression-project/blob/main/GettingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Source Type:** Custom-made Zillow API

Link for API: https://rapidapi.com/apimaker/api/zillow-com1/



---



In [ ]:
from google.colab import drive, files
import json 
import pandas as pd
import requests
import time

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)

In [ ]:
drive.mount('/drive', force_remount=False)

#keys =
city = 'New York'
state = 'NY'
search_query = city + ', ' + state
print(search_query)

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).
New York, NY


In [ ]:
api_key_df = pd.read_csv('/drive/MyDrive/df_to_csv/api_key.csv')
api_key = api_key_df.iloc[0,:].apply(str).values
api_key = ' '.join([str(i) for i in api_key]) 

In [ ]:
#FIRST PAGE
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":search_query,"home_type":"Apartments",
               "page":"1"}

headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key,
  "minPrice":'6000000',
  "sqftMin":'150'
}

response = requests.request("GET", url, headers=headers, params=querystring)
response = response.json()

In [ ]:
response

{'props': [{'address': '333 E 91st St APT 28-A, New York, NY 10128',
   'bathrooms': 3,
   'bedrooms': 3,
   'contingentListingType': None,
   'country': 'USA',
   'currency': 'USD',
   'dateSold': None,
   'daysOnZillow': -1,
   'hasImage': True,
   'imgSrc': 'https://photos.zillowstatic.com/fp/9a9763657e1ec9332a8d5906648c213e-p_e.jpg',
   'latitude': 40.780277,
   'listingDateTime': None,
   'listingStatus': 'FOR_SALE',
   'listingSubType': {'is_FSBA': True, 'is_openHouse': True},
   'livingArea': 1818,
   'longitude': -73.94728,
   'lotAreaUnit': None,
   'lotAreaValue': None,
   'price': 1995000,
   'propertyType': 'APARTMENT',
   'zpid': '123971408'},
  {'address': '677 Seneca Ave FLOOR 1, Ridgewood, NY 11385',
   'bathrooms': 4,
   'bedrooms': 5,
   'contingentListingType': None,
   'country': 'USA',
   'currency': 'USD',
   'dateSold': None,
   'daysOnZillow': -1,
   'hasImage': True,
   'imgSrc': 'https://photos.zillowstatic.com/fp/d3376a177ad744b2d9c661302e5ee725-p_e.jpg',
   

In [ ]:
pd.set_option('display.max_columns', 500)
df_first_page = pd.json_normalize(data=response['props'])
print('Columns -->', df_first_page.columns)
print('Nuw of Columns -->', len(df_first_page.columns))
print('Num of rows -->', len(df_first_page) )
df_first_page.head(2)

Columns --> Index(['dateSold', 'propertyType', 'lotAreaValue', 'address', 'daysOnZillow',
       'price', 'listingDateTime', 'longitude', 'latitude',
       'contingentListingType', 'listingStatus', 'zpid', 'imgSrc',
       'livingArea', 'bathrooms', 'lotAreaUnit', 'country', 'currency',
       'bedrooms', 'hasImage', 'listingSubType.is_FSBA',
       'listingSubType.is_openHouse'],
      dtype='object')
Nuw of Columns --> 22
Num of rows --> 40


,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,listingStatus,zpid,imgSrc,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse
0,None,APARTMENT,NaN,"333 E 91st St APT 28-A, New York, NY 10128",-1,1995000,None,-73.94728,40.780277,None,FOR_SALE,123971408,https://photos.zillowstatic.com/fp/9a9763657e1...,1818.0,3,None,USA,USD,3,True,True,True
1,None,APARTMENT,NaN,"677 Seneca Ave FLOOR 1, Ridgewood, NY 11385",-1,1588000,None,-73.90807,40.703300,None,FOR_SALE,2077855350,https://photos.zillowstatic.com/fp/d3376a177ad...,3000.0,4,None,USA,USD,5,True,True,NaN


In [ ]:
#EXTENDED SEARCH
zpids = df_first_page['zpid'].tolist()

detail_list = []
for zpid in zpids:
  url = "https://zillow-com1.p.rapidapi.com/property"

  querystring = {"zpid":zpid}

  headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key
  }

  response_zpids = requests.request("GET", url, headers=headers, params=querystring)
  response_zpids_json = response_zpids.json()
  time.sleep(6.1)

  detail_list.append(response_zpids_json)

In [ ]:
details_cols = [
      'propertyTaxRate',
      'timeOnZillow',
      'isListedByOwner',
      'zipcode',
      'annualHomeownersInsurance',
      'rentZestimate',
      'yearBuilt',
      'pageViewCount',
      'listingProvider.postingGroupName',
      'resoFacts.cityRegion',
      'resoFacts.hasCooling',
      'resoFacts.hasView']

df_prop_details = pd.json_normalize(data=detail_list)
df_prop_details = df_prop_details[details_cols]

df_first_page = pd.concat([df_first_page, df_prop_details], axis=1)
df_first_page.head(5)

,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,listingStatus,zpid,imgSrc,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse,propertyTaxRate,timeOnZillow,isListedByOwner,zipcode,annualHomeownersInsurance,rentZestimate,yearBuilt,pageViewCount,listingProvider.postingGroupName,resoFacts.cityRegion,resoFacts.hasCooling,resoFacts.hasView
0,None,APARTMENT,NaN,"333 E 91st St APT 28-A, New York, NY 10128",-1,1995000,None,-73.947280,40.780277,None,FOR_SALE,123971408,https://photos.zillowstatic.com/fp/9a9763657e1...,1818.0,3,None,USA,USD,3,True,True,True,0.88,174 days,False,10128,8379,12146.0,2007.0,3892,Compass,New York,True,False
1,None,APARTMENT,NaN,"677 Seneca Ave FLOOR 1, Ridgewood, NY 11385",-1,1588000,None,-73.908070,40.703300,None,FOR_SALE,2077855350,https://photos.zillowstatic.com/fp/d3376a177ad...,3000.0,4,None,USA,USD,5,True,True,NaN,0.84,18 hours,False,11385,6670,2894.0,1920.0,113,EXP Realty NYC,Ridgewood,True,False
2,None,APARTMENT,NaN,"474 48th Ave APT 29B, Long Island City, NY 11109",-1,735000,None,-73.956340,40.744408,None,FOR_SALE,79739498,https://photos.zillowstatic.com/fp/93f19aea44f...,1046.0,2,None,USA,USD,2,True,True,True,0.84,167 days,False,11109,3087,3319.0,1996.0,6591,Modern Spaces,Long Island City,False,True
3,None,APARTMENT,NaN,"771 E 9th St APT 1, Brooklyn, NY 11230",-1,1580000,None,-73.967850,40.630030,None,FOR_SALE,2065298549,https://photos.zillowstatic.com/fp/5848584357f...,5000.0,6,None,USA,USD,14,True,True,NaN,0.65,2 days,False,11230,6636,NaN,1925.0,254,Remax Edge,Brooklyn,False,False
4,None,APARTMENT,NaN,"4 W 21st St APT 11B, New York, NY 10010",-1,1750000,None,-73.990395,40.740055,None,FOR_SALE,72538269,https://photos.zillowstatic.com/fp/987d45ac953...,1674.0,3,None,USA,USD,3,True,True,NaN,0.88,476 days,False,10010,7350,11710.0,2006.0,2540,Brown Harris Stevens,New York,True,False


In [ ]:
#SECOND PAGE
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":search_query,"home_type":"Apartments",
               "page":"2"}

headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key,
  "minPrice":'6000000',
  "sqftMin":'150'
}

response = requests.request("GET", url, headers=headers, params=querystring)
response = response.json()

In [ ]:
response

{'props': [{'address': '30-06 42nd St #6FAM, Astoria, NY 11103',
   'bathrooms': 6,
   'bedrooms': 12,
   'contingentListingType': None,
   'country': 'USA',
   'currency': 'USD',
   'dateSold': None,
   'daysOnZillow': -1,
   'hasImage': True,
   'imgSrc': 'https://photos.zillowstatic.com/fp/81a8928ee462f9e3641f46ec5f28b22b-p_e.jpg',
   'latitude': 40.76307,
   'listingDateTime': None,
   'listingStatus': 'FOR_SALE',
   'listingSubType': {'is_FSBA': True},
   'livingArea': 6500,
   'longitude': -73.91379,
   'lotAreaUnit': None,
   'lotAreaValue': None,
   'price': 1595000,
   'propertyType': 'APARTMENT',
   'zpid': '2065665911'},
  {'address': '2205 Avenue X #1, Brooklyn, NY 11235',
   'bathrooms': 0,
   'bedrooms': 0,
   'contingentListingType': None,
   'country': 'USA',
   'currency': 'USD',
   'dateSold': None,
   'daysOnZillow': -1,
   'hasImage': None,
   'imgSrc': 'https://maps.googleapis.com/maps/api/staticmap?mobile=false&sensor=true&maptype=satellite&size=575x242&zoom=17&ce

In [ ]:
df_second_page = pd.json_normalize(data=response['props'])
print('Columns -->', df_second_page.columns)
print('Nuw of Columns -->', len(df_second_page.columns))
print('Num of rows -->', len(df_second_page) )
df_second_page.head(5)

Columns --> Index(['dateSold', 'propertyType', 'lotAreaValue', 'address', 'daysOnZillow',
       'price', 'listingDateTime', 'longitude', 'latitude',
       'contingentListingType', 'listingStatus', 'zpid', 'imgSrc',
       'livingArea', 'bathrooms', 'lotAreaUnit', 'country', 'currency',
       'bedrooms', 'hasImage', 'listingSubType.is_FSBA',
       'listingSubType.is_foreclosure', 'listingSubType.is_openHouse'],
      dtype='object')
Nuw of Columns --> 23
Num of rows --> 40


,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,listingStatus,zpid,imgSrc,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_FSBA,listingSubType.is_foreclosure,listingSubType.is_openHouse
0,None,APARTMENT,NaN,"30-06 42nd St #6FAM, Astoria, NY 11103",-1,1595000,None,-73.91379,40.763070,None,FOR_SALE,2065665911,https://photos.zillowstatic.com/fp/81a8928ee46...,6500.0,6,None,USA,USD,12,True,True,NaN,NaN
1,None,APARTMENT,NaN,"2205 Avenue X #1, Brooklyn, NY 11235",-1,699000,None,-73.94795,40.593250,None,FOR_SALE,2065418070,https://maps.googleapis.com/maps/api/staticmap...,2180.0,0,None,USA,USD,0,None,True,NaN,NaN
2,None,APARTMENT,NaN,"415 Woodward Ave SUITE 1, Ridgewood, NY 11385",-1,2000000,None,-73.91150,40.708580,None,FOR_SALE,2070765251,https://photos.zillowstatic.com/fp/f05f13909f2...,4500.0,5,None,USA,USD,11,True,True,NaN,NaN
3,None,APARTMENT,NaN,"555 Main St APT 1111, New York, NY 10044",-1,455000,None,-73.94948,40.762577,None,FOR_SALE,2098519057,https://photos.zillowstatic.com/fp/a68792d7720...,596.0,1,None,USA,USD,0,True,True,NaN,NaN
4,None,APARTMENT,NaN,"71 Coffey St #1, Brooklyn, NY 11231",-1,2995000,None,-74.01303,40.675236,None,FOR_SALE,2076670359,https://photos.zillowstatic.com/fp/c935cc2d699...,4193.0,4,None,USA,USD,6,True,True,NaN,NaN


In [ ]:
#EXTENDED SEARCH
zpids = df_second_page['zpid'].tolist()

detail_list = []
for zpid in zpids:
  url = "https://zillow-com1.p.rapidapi.com/property"

  querystring = {"zpid":zpid}

  headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key
  }

  response_zpids = requests.request("GET", url, headers=headers, params=querystring)
  response_zpids_json = response_zpids.json()
  time.sleep(6.1)

  detail_list.append(response_zpids_json)

In [ ]:
df_prop_details = pd.json_normalize(data=detail_list)
df_prop_details = df_prop_details[details_cols]

df_second_page = pd.concat([df_second_page, df_prop_details], axis=1)
df_second_page.head(5)

,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,listingStatus,zpid,imgSrc,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_FSBA,listingSubType.is_foreclosure,listingSubType.is_openHouse,propertyTaxRate,timeOnZillow,isListedByOwner,zipcode,annualHomeownersInsurance,rentZestimate,yearBuilt,pageViewCount,listingProvider.postingGroupName,resoFacts.cityRegion,resoFacts.hasCooling,resoFacts.hasView
0,None,APARTMENT,NaN,"30-06 42nd St #6FAM, Astoria, NY 11103",-1,1595000,None,-73.91379,40.763070,None,FOR_SALE,2065665911,https://photos.zillowstatic.com/fp/81a8928ee46...,6500.0,6,None,USA,USD,12,True,True,NaN,NaN,0.84,24 days,False,11103,6699,NaN,1920,1149,Modern Spaces,Astoria,False,False
1,None,APARTMENT,NaN,"2205 Avenue X #1, Brooklyn, NY 11235",-1,699000,None,-73.94795,40.593250,None,FOR_SALE,2065418070,https://maps.googleapis.com/maps/api/staticmap...,2180.0,0,None,USA,USD,0,None,True,NaN,NaN,0.65,5 days,False,11235,2936,NaN,1927,294,REMAX Elite,Brooklyn,False,False
2,None,APARTMENT,NaN,"415 Woodward Ave SUITE 1, Ridgewood, NY 11385",-1,2000000,None,-73.91150,40.708580,None,FOR_SALE,2070765251,https://photos.zillowstatic.com/fp/f05f13909f2...,4500.0,5,None,USA,USD,11,True,True,NaN,NaN,0.84,5 days,False,11385,8400,2049.0,1915,238,Continent Realty LLC,Ridgewood,False,False
3,None,APARTMENT,NaN,"555 Main St APT 1111, New York, NY 10044",-1,455000,None,-73.94948,40.762577,None,FOR_SALE,2098519057,https://photos.zillowstatic.com/fp/a68792d7720...,596.0,1,None,USA,USD,0,True,True,NaN,NaN,0.88,20 days,False,10044,1911,2984.0,1975,1874,Houlihan Lawrence,New York,True,False
4,None,APARTMENT,NaN,"71 Coffey St #1, Brooklyn, NY 11231",-1,2995000,None,-74.01303,40.675236,None,FOR_SALE,2076670359,https://photos.zillowstatic.com/fp/c935cc2d699...,4193.0,4,None,USA,USD,6,True,True,NaN,NaN,0.65,17 days,False,11231,12579,3240.0,2005,1257,Compass,Brooklyn,True,False


In [ ]:
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":search_query,"home_type":"Apartments",
               "page":"3"}

headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key,
  "minPrice":'6000000',
  "sqftMin":'150'
}

response = requests.request("GET", url, headers=headers, params=querystring)
response = response.json()

In [ ]:
pd.set_option('display.max_columns', 500)
df_third_page = pd.json_normalize(data=response['props'])
print('Columns -->', df_third_page.columns)
print('Nuw of Columns -->', len(df_third_page.columns))
print('Num of rows -->', len(df_third_page) )
df_third_page.head(5)

Columns --> Index(['dateSold', 'propertyType', 'lotAreaValue', 'address', 'daysOnZillow',
       'price', 'listingDateTime', 'longitude', 'latitude',
       'contingentListingType', 'listingStatus', 'zpid', 'imgSrc',
       'livingArea', 'bathrooms', 'lotAreaUnit', 'country', 'currency',
       'bedrooms', 'hasImage', 'listingSubType.is_FSBA',
       'listingSubType.is_openHouse'],
      dtype='object')
Nuw of Columns --> 22
Num of rows --> 40


,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,listingStatus,zpid,imgSrc,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse
0,None,APARTMENT,NaN,"995 5th Ave APT 3N, New York, NY 10028",-1,10900000,None,-73.962550,40.778004,None,FOR_SALE,82227436,https://photos.zillowstatic.com/fp/566294aa291...,4118.0,5,None,USA,USD,4,True,True,NaN
1,None,APARTMENT,NaN,"520 W 23rd St APT 5E, New York, NY 10011",-1,730000,None,-74.004814,40.747807,None,FOR_SALE,72514862,https://photos.zillowstatic.com/fp/88880419271...,608.0,1,None,USA,USD,1,True,True,True
2,None,APARTMENT,NaN,"2 W End Ave APT 2C, Brooklyn, NY 11235",-1,435000,None,-73.954160,40.581326,None,FOR_SALE,245021492,https://photos.zillowstatic.com/fp/d4758dd3a48...,1150.0,1,None,USA,USD,2,True,True,NaN
3,None,APARTMENT,NaN,"110-45 Queens Blvd #1A, Forest Hills, NY 11375",-1,399000,None,-73.838830,40.719513,None,FOR_SALE,124449232,https://photos.zillowstatic.com/fp/24eff940eb2...,NaN,2,None,USA,USD,0,True,True,NaN
4,None,APARTMENT,0.0,"403 Willow Rd E #1, Staten Island, NY 10314",-1,435000,None,-74.152290,40.615310,None,FOR_SALE,32303401,https://photos.zillowstatic.com/fp/bf845effe0b...,1248.0,2,sqft,USA,USD,2,True,True,NaN


In [ ]:
zpids = df_third_page['zpid'].tolist()

detail_list = []
for zpid in zpids:
  url = "https://zillow-com1.p.rapidapi.com/property"

  querystring = {"zpid":zpid}

  headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key
  }

  response_zpids = requests.request("GET", url, headers=headers, params=querystring)
  response_zpids_json = response_zpids.json()
  time.sleep(6.1)

  detail_list.append(response_zpids_json)

In [ ]:
df_prop_details = pd.json_normalize(data=detail_list)
df_prop_details = df_prop_details[details_cols]

df_third_page = pd.concat([df_third_page, df_prop_details], axis=1)
df_third_page.head(5)

,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,listingStatus,zpid,imgSrc,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse,propertyTaxRate,timeOnZillow,isListedByOwner,zipcode,annualHomeownersInsurance,rentZestimate,yearBuilt,pageViewCount,listingProvider.postingGroupName,resoFacts.cityRegion,resoFacts.hasCooling,resoFacts.hasView
0,None,APARTMENT,NaN,"995 5th Ave APT 3N, New York, NY 10028",-1,10900000,None,-73.962550,40.778004,None,FOR_SALE,82227436,https://photos.zillowstatic.com/fp/566294aa291...,4118.0,5,None,USA,USD,4,True,True,NaN,0.88,141 days,False,10028,45780,14990.0,1926,679,Sotheby's International Realty,New York,True,False
1,None,APARTMENT,NaN,"520 W 23rd St APT 5E, New York, NY 10011",-1,730000,None,-74.004814,40.747807,None,FOR_SALE,72514862,https://photos.zillowstatic.com/fp/88880419271...,608.0,1,None,USA,USD,1,True,True,True,0.88,78 days,False,10011,3066,5450.0,2003,511,Compass,New York,True,False
2,None,APARTMENT,NaN,"2 W End Ave APT 2C, Brooklyn, NY 11235",-1,435000,None,-73.954160,40.581326,None,FOR_SALE,245021492,https://photos.zillowstatic.com/fp/d4758dd3a48...,1150.0,1,None,USA,USD,2,True,True,NaN,0.65,178 days,False,11235,1827,2232.0,1953,427,"Diamond Real, Inc.",Brooklyn,False,False
3,None,APARTMENT,NaN,"110-45 Queens Blvd #1A, Forest Hills, NY 11375",-1,399000,None,-73.838830,40.719513,None,FOR_SALE,124449232,https://photos.zillowstatic.com/fp/24eff940eb2...,NaN,2,None,USA,USD,0,True,True,NaN,0.84,6 days,False,11375,1676,2400.0,1951,299,Century 21 Amiable Realty Group II Inc,Forest Hills,False,False
4,None,APARTMENT,0.0,"403 Willow Rd E #1, Staten Island, NY 10314",-1,435000,None,-74.152290,40.615310,None,FOR_SALE,32303401,https://photos.zillowstatic.com/fp/bf845effe0b...,1248.0,2,sqft,USA,USD,2,True,True,NaN,0.87,36 days,False,10314,1827,1381.0,1977,1351,NaN,Staten Island,True,False


In [ ]:
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":search_query,"home_type":"Apartments",
               "page":"4"}

headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key,
  "minPrice":'6000000',
  "sqftMin":'150'
}

response = requests.request("GET", url, headers=headers, params=querystring)
response = response.json()

In [ ]:
pd.set_option('display.max_columns', 500)
df_fourth_page = pd.json_normalize(data=response['props'])
print('Columns -->', df_fourth_page.columns)
print('Nuw of Columns -->', len(df_fourth_page.columns))
print('Num of rows -->', len(df_fourth_page) )
df_fourth_page.head(5)

Columns --> Index(['dateSold', 'propertyType', 'lotAreaValue', 'address', 'daysOnZillow',
       'price', 'listingDateTime', 'longitude', 'latitude',
       'contingentListingType', 'listingStatus', 'zpid', 'imgSrc',
       'livingArea', 'bathrooms', 'lotAreaUnit', 'country', 'currency',
       'bedrooms', 'hasImage', 'listingSubType.is_FSBA',
       'listingSubType.is_openHouse'],
      dtype='object')
Nuw of Columns --> 22
Num of rows --> 40


,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,listingStatus,zpid,imgSrc,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse
0,None,APARTMENT,NaN,"2 W End Ave APT 2C, Brooklyn, NY 11235",-1,435000,None,-73.954160,40.581326,None,FOR_SALE,245021492,https://photos.zillowstatic.com/fp/d4758dd3a48...,1150.0,1,None,USA,USD,2,True,True,NaN
1,None,APARTMENT,NaN,"99-40 63rd Rd #5T, Rego Park, NY 11374",-1,385000,None,-73.855515,40.732310,None,FOR_SALE,2066958535,https://photos.zillowstatic.com/fp/80b5bbe4035...,NaN,1,None,USA,USD,2,True,True,True
2,None,APARTMENT,NaN,"7005 13th Ave APT 1, Brooklyn, NY 11228",-1,1250000,None,-74.004450,40.622850,None,FOR_SALE,2065461408,https://photos.zillowstatic.com/fp/acdb57467cb...,NaN,2,None,USA,USD,3,True,True,NaN
3,None,APARTMENT,NaN,"215 E 96th St #32C, New York, NY 10128",-1,1195000,None,-73.947970,40.785130,None,FOR_SALE,321915286,https://photos.zillowstatic.com/fp/b7ba5075ef4...,1183.0,2,None,USA,USD,2,True,True,True
4,None,APARTMENT,NaN,"242 E 25th St APT 10A, New York, NY 10010",-1,675000,None,-73.980550,40.739170,None,FOR_SALE,321905366,https://photos.zillowstatic.com/fp/2d9df910ab9...,720.0,1,None,USA,USD,1,True,True,NaN


In [ ]:
zpids = df_fourth_page['zpid'].tolist()

detail_list = []
for zpid in zpids:
  url = "https://zillow-com1.p.rapidapi.com/property"

  querystring = {"zpid":zpid}

  headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key
  }

  response_zpids = requests.request("GET", url, headers=headers, params=querystring)
  response_zpids_json = response_zpids.json()
  time.sleep(6.1)

  detail_list.append(response_zpids_json)

In [ ]:
df_prop_details = pd.json_normalize(data=detail_list)
df_prop_details = df_prop_details[details_cols]

df_fourth_page = pd.concat([df_fourth_page, df_prop_details], axis=1)
df_third_page.head(5)

,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,listingStatus,zpid,imgSrc,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse,propertyTaxRate,timeOnZillow,isListedByOwner,zipcode,annualHomeownersInsurance,rentZestimate,yearBuilt,pageViewCount,listingProvider.postingGroupName,resoFacts.cityRegion,resoFacts.hasCooling,resoFacts.hasView
0,None,APARTMENT,NaN,"995 5th Ave APT 3N, New York, NY 10028",-1,10900000,None,-73.962550,40.778004,None,FOR_SALE,82227436,https://photos.zillowstatic.com/fp/566294aa291...,4118.0,5,None,USA,USD,4,True,True,NaN,0.88,141 days,False,10028,45780,14990.0,1926,679,Sotheby's International Realty,New York,True,False
1,None,APARTMENT,NaN,"520 W 23rd St APT 5E, New York, NY 10011",-1,730000,None,-74.004814,40.747807,None,FOR_SALE,72514862,https://photos.zillowstatic.com/fp/88880419271...,608.0,1,None,USA,USD,1,True,True,True,0.88,78 days,False,10011,3066,5450.0,2003,511,Compass,New York,True,False
2,None,APARTMENT,NaN,"2 W End Ave APT 2C, Brooklyn, NY 11235",-1,435000,None,-73.954160,40.581326,None,FOR_SALE,245021492,https://photos.zillowstatic.com/fp/d4758dd3a48...,1150.0,1,None,USA,USD,2,True,True,NaN,0.65,178 days,False,11235,1827,2232.0,1953,427,"Diamond Real, Inc.",Brooklyn,False,False
3,None,APARTMENT,NaN,"110-45 Queens Blvd #1A, Forest Hills, NY 11375",-1,399000,None,-73.838830,40.719513,None,FOR_SALE,124449232,https://photos.zillowstatic.com/fp/24eff940eb2...,NaN,2,None,USA,USD,0,True,True,NaN,0.84,6 days,False,11375,1676,2400.0,1951,299,Century 21 Amiable Realty Group II Inc,Forest Hills,False,False
4,None,APARTMENT,0.0,"403 Willow Rd E #1, Staten Island, NY 10314",-1,435000,None,-74.152290,40.615310,None,FOR_SALE,32303401,https://photos.zillowstatic.com/fp/bf845effe0b...,1248.0,2,sqft,USA,USD,2,True,True,NaN,0.87,36 days,False,10314,1827,1381.0,1977,1351,NaN,Staten Island,True,False


In [ ]:
#concating all dataframes
df = pd.concat([df_first_page, df_second_page, df_third_page, df_fourth_page],axis=0)
df

,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,listingStatus,zpid,imgSrc,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse,propertyTaxRate,timeOnZillow,isListedByOwner,zipcode,annualHomeownersInsurance,rentZestimate,yearBuilt,pageViewCount,listingProvider.postingGroupName,resoFacts.cityRegion,resoFacts.hasCooling,resoFacts.hasView,listingSubType.is_foreclosure
0,None,APARTMENT,NaN,"333 E 91st St APT 28-A, New York, NY 10128",-1,1995000,None,-73.947280,40.780277,None,FOR_SALE,123971408,https://photos.zillowstatic.com/fp/9a9763657e1...,1818.0,3,None,USA,USD,3,True,True,True,0.88,174 days,False,10128,8379,12146.0,2007.0,3892,Compass,New York,True,False,NaN
1,None,APARTMENT,NaN,"677 Seneca Ave FLOOR 1, Ridgewood, NY 11385",-1,1588000,None,-73.908070,40.703300,None,FOR_SALE,2077855350,https://photos.zillowstatic.com/fp/d3376a177ad...,3000.0,4,None,USA,USD,5,True,True,NaN,0.84,18 hours,False,11385,6670,2894.0,1920.0,113,EXP Realty NYC,Ridgewood,True,False,NaN
2,None,APARTMENT,NaN,"474 48th Ave APT 29B, Long Island City, NY 11109",-1,735000,None,-73.956340,40.744408,None,FOR_SALE,79739498,https://photos.zillowstatic.com/fp/93f19aea44f...,1046.0,2,None,USA,USD,2,True,True,True,0.84,167 days,False,11109,3087,3319.0,1996.0,6591,Modern Spaces,Long Island City,False,True,NaN
3,None,APARTMENT,NaN,"771 E 9th St APT 1, Brooklyn, NY 11230",-1,1580000,None,-73.967850,40.630030,None,FOR_SALE,2065298549,https://photos.zillowstatic.com/fp/5848584357f...,5000.0,6,None,USA,USD,14,True,True,NaN,0.65,2 days,False,11230,6636,NaN,1925.0,254,Remax Edge,Brooklyn,False,False,NaN
4,None,APARTMENT,NaN,"4 W 21st St APT 11B, New York, NY 10010",-1,1750000,None,-73.990395,40.740055,None,FOR_SALE,72538269,https://photos.zillowstatic.com/fp/987d45ac953...,1674.0,3,None,USA,USD,3,True,True,NaN,0.88,476 days,False,10010,7350,11710.0,2006.0,2540,Brown Harris Stevens,New York,True,False,NaN
5,None,APARTMENT,NaN,"4-74 48th Ave PENTHOUSE 1C, Long Island City, ...",-1,555555,None,-73.956340,40.744408,None,FOR_SALE,219836786,https://photos.zillowstatic.com/fp/df7f9142c3c...,921.0,1,None,USA,USD,1,True,True,True,0.84,631 days,False,11109,2333,3776.0,1996.0,946,Modern Spaces,Long Island City,True,True,NaN
6,None,APARTMENT,NaN,"4-74 48th Ave APT 19A, Long Island City, NY 11109",-1,699999,None,-73.956340,40.744408,None,FOR_SALE,113197815,https://photos.zillowstatic.com/fp/27bea9f710f...,1046.0,2,None,USA,USD,2,True,True,True,0.84,217 days,False,11109,2940,2900.0,1996.0,3573,Modern Spaces,Long Island City,False,False,NaN
7,None,APARTMENT,NaN,"300 E 40th St APT 8F, New York, NY 10016",-1,720000,None,-73.973140,40.748100,None,FOR_SALE,244884473,https://photos.zillowstatic.com/fp/592f2685862...,NaN,1,None,USA,USD,0,True,True,NaN,0.88,1 hour,False,10016,3024,3866.0,1967.0,3,Brown Harris Stevens,New York,False,False,NaN
8,None,APARTMENT,NaN,"4-74 48th Ave #21AB, Long Island City, NY 11109",-1,1250000,None,-73.956800,40.744200,None,FOR_SALE,2078556904,https://photos.zillowstatic.com/fp/6a9ac7ac0b6...,2200.0,2,None,USA,USD,3,True,True,True,0.84,8 days,False,11109,5250,5351.0,1996.0,1658,Compass,Long Island City,False,False,NaN
9,None,APARTMENT,1077.0,"310 E 46th St APT 23G, New York, NY 10017",-1,1250000,None,-73.969730,40.752007,None,FOR_SALE,82228315,https://photos.zillowstatic.com/fp/10317ca847f...,1077.0,1,sqft,USA,USD,1,True,True,True,0.88,2 days,False,10017,5250,4440.0,1929.0,160,SERHANT.,New York,True,True,NaN


In [ ]:
df.to_csv('/drive/MyDrive/df_to_csv/zillow_df.csv', encoding='utf-8', index=False) # saving df on my own Google Drive